In [ ]:
from functions.streamlitfunc import *
from functions.utils import *
import time
import streamlit as st
from functions.chat_bot import *
from functions.utils import local_settings
from streamlit_extras.switch_page_button import switch_page 
# Gather the information provided by the user fill them in the following functions:
# - get_recommendation
# Use the output of the function and present the recommendation in the following format:
# - Restaurant Name
# - Cuisine Type
# - Average Price
# - Location
# - Restaurant Style
# - Restaurant Schedule
# Ask if the user would like to see either see another recommendation, proceed with a reservation \
# at the suggested restaurant or know more about the restaurant.


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from prompts_list import *
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from functions.utils import *
from openai import OpenAI

In [ ]:
files = ['data\CP-23Group4 Project Proposal.pdf', 'data\Food Personalities.pdf']
fil = Filomena()
fil.load_documents(files)

In [ ]:
fil.get_completion('[Identification] Hello.', identified=True)

In [ ]:
for message in fil.messages:
    print(message['content'])

In [ ]:
for message in fil.messages:
    if message["role"] == "user":
        avatar = 'https://miro.medium.com/v2/resize:fit:1100/format:webp/1*_ARzR7F_fff_KI14yMKBzw.png'
        print('user')
    else:
        avatar = avatar="https://cdn.discordapp.com/attachments/1150843302644547768/1190661589347602492/1000_F_378272550_xN8H7ZVudgCYWzfuZxRxVS5uFKjzsoMg.jpg?ex=65a29d04&is=65902804&hm=4a84c24f579a1d8ac5493b28f47b50c1dc7aaabc2832cb090bd7e4e95b2ab786&"
            # with st.chat_message(message["role"], avatar=avatar):
            #     st.markdown(message["content"])

In [ ]:
fil.prep_question('Who are the authors of FlavourFlix?', 
                  "Fellow Foodie, I'm Filomena, your virtual assistant from FlavourFlix! We specialize in recommending restaurants based on your preferences and requirements. So, what are you in the mood for today? Let's find the perfect restaurant for you!")

In [ ]:
from functions.filomena_utils import *
query_helper = GPT_Helper(OPENAI_API_KEY=local_settings.OPENAI_API_KEY, system_behavior=prompt_templates['Query Helper System'])

In [ ]:
def prep_question( query, chatbot_message):
    prompt = f"""
            The User Message is:{query}
            The ChatBot Message is:{chatbot_message}
            """

    query_formatted = query_helper.get_completion(prompt)
    return query_formatted

In [17]:
from functions.utils import *
from functions.chat_bot import *

In [18]:
fil = Filomena()
fil.intialize_filomena(files=['data\CP-23Group4 Project Proposal.pdf', 'data\Food Personalities.pdf'])

In [19]:
fil.messages

[{'role': 'assistant',
  'content': "Hello! Welcome to FlavourFlix, your go-to virtual assistant for restaurant recommendations. I'm Filomena, here to help you find the perfect dining experience. How can I assist you today?"}]

In [ ]:
fil.generate_response('Who are the authors of FlavourFlix?')

In [ ]:
fil.generate_response('What is the purpose of FlavourFlix?')

In [20]:
fil.generate_response('What are the three most expensive restaurants in Albufeira?')

"AI: Thank you for your question! I'm currently searching for the three most expensive restaurants in Albufeira. Please bear with me while I gather the information for you."

In [86]:
from langchain.agents import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [87]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [88]:
@tool
def get_restaurant_info(restaurant_name):
    """Gathers all the restaurant information from the FlavourFlix restaurant database, based on the name of the restaurant."""
    restaurant_data = pd.read_csv('data\preprocessed_restaurant_data.csv')
    restaurant_data = restaurant_data[restaurant_data['name'] == restaurant_name]
    result = f"""Name: {restaurant_data["name"].values[0]} \n /
    Cuisine Type: {restaurant_data["cuisine"].values[0]} \n /
    Average Price: {restaurant_data["averagePrice"].values[0]} \n /
    Location: {restaurant_data["location"].values[0]} \n /
    Restaurant Style: {restaurant_data["style"].values[0]} \n /
    Restaurant Schedule: {restaurant_data["schedule"].values[0]}"""
    return result

In [89]:
client_data = pd.read_csv('data\clientData.csv')

In [62]:
# !python -m spacy download en_core_web_md

In [90]:
import spacy
from sklearn.metrics.pairwise import cosine_similarity 
def get_data_match(data, word, col_to_match):
    nlp = spacy.load("en_core_web_md")

    word_embedding = nlp(word).vector
    word_embedding = nlp('mexican').vector
    similarities = {}
    for token in list(data[col_to_match].unique()):
        token_embedding = nlp(token).vector
        similarities[token] = cosine_similarity([word_embedding], [token_embedding])[0][0]

    return max(similarities, key=similarities.get)

In [118]:
def filter_schedule(restaurants, dinner_hour = None, lunch_hour = None):
    def contains_time_interval(schedule):
        if schedule in ['Closed', 'Not Available']:
            return False
        schedule_dict = {}
        try:
            schedule_dict = ast.literal_eval(schedule)
        except (SyntaxError, ValueError):
            pass

        for day_schedule in schedule_dict.values():
            if day_schedule == 'Closed':
                continue
            times = day_schedule.split(', ')
            for time in times:
                start, end = map(pd.to_datetime, time.split(' - '))
                if dinner_hour and lunch_hour:
                    if (start <= dinner_start <= end) or (start <= dinner_end <= end) or (start <= lunch_start <= end) or (start <= lunch_end <= end):
                        return True
                elif dinner_hour:
                    if (start <= dinner_start <= end) or (start <= dinner_end <= end):
                        return True
                elif lunch_hour:
                    if (start <= lunch_start <= end) or (start <= lunch_end <= end):
                        return True
                else:
                    return False
        return False

    dinner_start, dinner_end = map(pd.to_datetime, dinner_hour.split(' - '))
    lunch_start, lunch_end = map(pd.to_datetime, lunch_hour.split(' - '))

    filtered_restaurants = []
    for index, row in restaurants.iterrows():
        if contains_time_interval(row['schedule']):
            filtered_restaurants.append(row['name'])

    return filtered_restaurants

In [127]:
from functions.location import *
from functions.preprocessement import *
from langdetect import detect

def get_recommendation(nationality=None, city=None, travel_car=None,  favourite_food=None,  restaurant_style=None,
                       cuisine_type=None, lunch_hour=None, dinner_hour=None,
                       normal_price_range=None):
    """Recommends restaurants based on the user's preferences and requirements."""
    restaurant_data = pd.read_csv('data/preprocessed_restaurant_data.csv')
    filtered_data = restaurant_data.copy()

    conditions = []

    #Location filters such that the restaurant is near the user and the place the user wants to go
    if city:
        city_match = get_data_match(restaurant_data, city, 'city')
        filtered_data = filtered_data[filtered_data['city'] == city_match]
        latitude, longitude = find_coordinates(city_match)
        if latitude is not None and longitude is not None:
            desired_location = Location(latitude, longitude)
            filtered_data = nearYou(desired_location, filtered_data)
            if travel_car:
                filtered_data['minutes_away'] = filtered_data.apply(lambda row: desired_location.getDirections(row['latitude'], row['longitude'], ['driving'])['driving'].minutes, axis=1)
                filtered_data = filtered_data[filtered_data['minutes_away'] <= 35]
            else:
                filtered_data['minutes_away'] = filtered_data.apply(lambda row: desired_location.getDirections(row['latitude'], row['longitude'], ['walking'])['walking'].minutes, axis=1)
                filtered_data = filtered_data[filtered_data['minutes_away'] <= 59]

    #Time filters such that the restaurant is open at the time the user wants to go
    if lunch_hour:
        filtered_data = filter_schedule(filtered_data, dinner_hour, lunch_hour)
    if dinner_hour:
        filtered_data = filter_schedule(filtered_data, dinner_hour, lunch_hour)
    
    #Additional filters
    if nationality and len(filtered_data) != 0:
        nationality_match = get_data_match(restaurant_data, nationality, 'cuisine')
        conditions.append(restaurant_data['cuisine'] == nationality_match)
    
    if restaurant_style:
        restaurant_style_match = get_data_match(restaurant_data, restaurant_style, 'style')
        conditions.append(restaurant_data['style'] == restaurant_style_match)
    
    if cuisine_type:
        cuisine_type_match = get_data_match(restaurant_data, cuisine_type, 'cuisine')
        conditions.append(restaurant_data['cuisine'] == cuisine_type_match)

    if normal_price_range:
        conditions.append(restaurant_data['averagePrice'] <= normal_price_range+4)

    if favourite_food:
        if detect(favourite_food) == 'en':
            conditions.append(restaurant_data['menu_en'].str.contains(favourite_food))
        else:
            conditions.append(restaurant_data['menu_pre_proc'].str.contains(favourite_food))
    

    # Combine conditions with logical OR
    if conditions and len(filtered_data) != 0:
        filtered_data = restaurant_data[pd.concat(conditions, axis=1).any(axis=1)]
    else:
        filtered_data = restaurant_data 
    

    # Ranking - sort by ratings or other criteria
    restaurant_data = restaurant_data.sort_values(by='ratingValue', ascending=False)

    # Return top recommendations
    return restaurant_data.head(5) 


In [128]:
get_recommendation(city='Albufeira', normal_price_range=20, favourite_food='massa')

,url,name,address,photo,averagePrice,chefName1,chefName2,chefName3,cuisine,michelin,...,city,ambienceRatingSummary,foodRatingSummary,serviceRatingSummary,paymentAcceptedSummary,outdoor_area,current_occupation,menu_pre_proc,menu_en,menu_pt
1324,https://www.thefork.com/restaurant/rei-da-gula...,Rei da Gula,"R. de Ceuta 63,Porto, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,15,Not Applicable,Not Applicable,Not Applicable,Pizzeria,0,...,Porto,NaN,NaN,NaN,"{'Credit Card', 'MBWay'}",0,0,{'Entradas - para começar ': {'Sopa': {'price'...,{'Starters - to get started': {'Sopa': {'price...,{'Entradas - para começar ': {'Soup': {'price'...
224,https://www.thefork.com/restaurant/caravela-wy...,Caravela - Wyndham Grand Algarve,"Av. André Jordan 39,Almancil, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,21,Not Applicable,Not Applicable,Not Applicable,International,0,...,Faro,10.0,10.0,10.0,"{'Credit Card', 'MBWay'}",0,23,{'Entradas': {'Creme De Couve-Flor Com Aroma D...,{'Appetizer': {'Cauliflower Cream with Wasabi ...,{'Starters': {'Creme De Couve-Flor Com Aroma D...
372,https://www.thefork.com/restaurant/alegre-me-h...,Alegre-me Healthy Brunch,"R. Antão Girão 77,Setúbal, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,12,Not Applicable,Not Applicable,Not Applicable,Vegetarian,0,...,Setúbal,10.0,10.0,10.0,{'Credit Card'},0,0,"{'Quesadilhas': {'Surfie': {'price': 9.5, 'des...","{'Quesadillas': {'Surfie': {'price': 9.5, 'des...","{'Quesadilhas': {'Surfista': {'price': 9.5, 'd..."
217,https://www.thefork.com/restaurant/grandbar-lo...,Grandbar & Lounge - Wyndham Grand Algarve,"Av. André Jordan 39,Almancil, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,14,Not Applicable,Not Applicable,Not Applicable,European,0,...,Faro,10.0,10.0,10.0,"{'Credit Card', 'MBWay'}",0,0,{'Bites wga': {'Mini espetadas frango \\\\Yaki...,{'Bites wga': {'Mini chicken kebabs \\\\Yakito...,{'Mordidas wga': {'Mini espetadas frango \\\\Y...
419,https://www.thefork.com/restaurant/casa-do-arc...,Casa do Arco by Douro Exclusive,"5085-252,São Cristovão do Douro, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,35,Pedro Pires,Not Applicable,Not Applicable,European,0,...,Vila Real,9.9,9.6,10.0,"{'Mastercard', 'Credit Card', 'Visa', 'Visa El...",0,0,{'Para Começar': {'Torradas De Pão': {'price':...,"{'To begin': {'Bread Toast': {'price': 8.0, 'd...",{'Para Começar': {'Torradas De Pão': {'price':...


In [33]:
restaurant_data = pd.read_csv('data\preprocessed_restaurant_data.csv')

In [98]:
import pandas as pd
import ast

def filter_schedule(restaurants, dinner_hour, lunch_hour):
    def contains_time_interval(schedule):
        if schedule in ['Closed', 'Not Available']:
            return False
        schedule_dict = {}
        try:
            schedule_dict = ast.literal_eval(schedule)
        except (SyntaxError, ValueError):
            pass

        for day_schedule in schedule_dict.values():
            if day_schedule == 'Closed':
                continue
            times = day_schedule.split(', ')
            for time in times:
                start, end = map(pd.to_datetime, time.split(' - '))
                if (start <= dinner_start <= end) or (start <= dinner_end <= end) or (start <= lunch_start <= end) or (start <= lunch_end <= end):
                    return True
        return False

    dinner_start, dinner_end = map(pd.to_datetime, dinner_hour.split(' - '))
    lunch_start, lunch_end = map(pd.to_datetime, lunch_hour.split(' - '))

    filtered_restaurants = []
    for index, row in restaurants.iterrows():
        if contains_time_interval(row['schedule']):
            filtered_restaurants.append(row['name'])

    return filtered_restaurants


In [91]:
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')
agent= initialize_agent( [get_restaurant_info],
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = False)

In [92]:
result = agent("Tell me about restaurant 'Chutneys'")

In [93]:
result

{'input': "Tell me about restaurant 'Chutneys'",
 'output': "The restaurant 'Chutneys' is an Indian restaurant located in Albufeira. It has an average price of 14 and follows a Festivities restaurant style. The restaurant is open from 11:30 AM to 2:30 PM and from 6:00 PM to 11:59 PM every day of the week."}

In [31]:
try:
    result = agent("Tell me abour restaurant 'A Galinha da Vizinha' in Albufeira")
except:
    print("exception on external access")



> Entering new AgentExecutor chain...
Question: What is the information about the restaurant 'A Galinha da Vizinha' in Albufeira?
Thought: I should use the get_restaurant_info tool to gather the information about the restaurant.
Action:
```
{
  "action": "get_restaurant_info",
  "action_input": "A Galinha da Vizinha"
}
```
exception on external access


In [4]:
from functions.location import *
from functions.preprocessement import *
from langdetect import detect
from langchain import hub
from sklearn.metrics.pairwise import cosine_similarity 
import spacy
from langchain.agents import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

@tool
def get_restaurant_info(restaurant_name):
    """Gathers all the restaurant information from the FlavourFlix restaurant database, based on the name of the restaurant."""
    restaurant_data = pd.read_csv('data\preprocessed_restaurant_data.csv')
    restaurant_data = restaurant_data[restaurant_data['name'] == restaurant_name]
    result = f"""Name: {restaurant_data["name"].values[0]} \n /
    Cuisine Type: {restaurant_data["cuisine"].values[0]} \n /
    Average Price: {restaurant_data["averagePrice"].values[0]} \n /
    Location: {restaurant_data["location"].values[0]} \n /
    Restaurant Style: {restaurant_data["style"].values[0]} \n /
    Restaurant Schedule: {restaurant_data["schedule"].values[0]}"""
    return result


def get_data_match(data, word, col_to_match):
    nlp = spacy.load("en_core_web_md")

    word_embedding = nlp(word).vector
    word_embedding = nlp('mexican').vector
    similarities = {}
    for token in list(data[col_to_match].unique()):
        token_embedding = nlp(token).vector
        similarities[token] = cosine_similarity([word_embedding], [token_embedding])[0][0]

    return max(similarities, key=similarities.get)


def filter_schedule(restaurants, dinner_hour = None, lunch_hour = None):
    def contains_time_interval(schedule):
        if schedule in ['Closed', 'Not Available']:
            return False
        schedule_dict = {}
        try:
            schedule_dict = ast.literal_eval(schedule)
        except (SyntaxError, ValueError):
            pass

        for day_schedule in schedule_dict.values():
            if day_schedule == 'Closed':
                continue
            times = day_schedule.split(', ')
            for time in times:
                start, end = map(pd.to_datetime, time.split(' - '))
                if dinner_hour and lunch_hour:
                    if (start <= dinner_start <= end) or (start <= dinner_end <= end) or (start <= lunch_start <= end) or (start <= lunch_end <= end):
                        return True
                elif dinner_hour:
                    if (start <= dinner_start <= end) or (start <= dinner_end <= end):
                        return True
                elif lunch_hour:
                    if (start <= lunch_start <= end) or (start <= lunch_end <= end):
                        return True
                else:
                    return False
        return False

    dinner_start, dinner_end = map(pd.to_datetime, dinner_hour.split(' - '))
    lunch_start, lunch_end = map(pd.to_datetime, lunch_hour.split(' - '))

    filtered_restaurants = []
    for index, row in restaurants.iterrows():
        if contains_time_interval(row['schedule']):
            filtered_restaurants.append(row['name'])

    return filtered_restaurants

@tool
def get_recommendation(nationality=None, city=None, travel_car=None,  favourite_food=None,  restaurant_style=None,
                       cuisine_type=None, lunch_hour=None, dinner_hour=None,
                       normal_price_range=None):
    """Recommends restaurants based on the user's preferences and requirements."""
    restaurant_data = pd.read_csv('data/preprocessed_restaurant_data.csv')
    filtered_data = restaurant_data.copy()

    conditions = []

    #Location filters such that the restaurant is near the user and the place the user wants to go
    if city:
        city_match = get_data_match(restaurant_data, city, 'city')
        filtered_data = filtered_data[filtered_data['city'] == city_match]
        latitude, longitude = find_coordinates(city_match)
        if latitude is not None and longitude is not None:
            desired_location = Location(latitude, longitude)
            filtered_data = nearYou(desired_location, filtered_data)
            if travel_car:
                filtered_data['minutes_away'] = filtered_data.apply(lambda row: desired_location.getDirections(row['latitude'], row['longitude'], ['driving'])['driving'].minutes, axis=1)
                filtered_data = filtered_data[filtered_data['minutes_away'] <= 35]
            else:
                filtered_data['minutes_away'] = filtered_data.apply(lambda row: desired_location.getDirections(row['latitude'], row['longitude'], ['walking'])['walking'].minutes, axis=1)
                filtered_data = filtered_data[filtered_data['minutes_away'] <= 59]

    #Time filters such that the restaurant is open at the time the user wants to go
    if lunch_hour:
        filtered_data = filter_schedule(filtered_data, dinner_hour, lunch_hour)
    if dinner_hour:
        filtered_data = filter_schedule(filtered_data, dinner_hour, lunch_hour)
    
    #Additional filters
    if nationality and len(filtered_data) != 0:
        nationality_match = get_data_match(restaurant_data, nationality, 'cuisine')
        conditions.append(restaurant_data['cuisine'] == nationality_match)
    
    if restaurant_style:
        restaurant_style_match = get_data_match(restaurant_data, restaurant_style, 'style')
        conditions.append(restaurant_data['style'] == restaurant_style_match)
    
    if cuisine_type:
        cuisine_type_match = get_data_match(restaurant_data, cuisine_type, 'cuisine')
        conditions.append(restaurant_data['cuisine'] == cuisine_type_match)

    if normal_price_range:
        conditions.append(restaurant_data['averagePrice'] <= normal_price_range+4)

    if favourite_food:
        if detect(favourite_food) == 'en':
            conditions.append(restaurant_data['menu_en'].str.contains(favourite_food))
        else:
            conditions.append(restaurant_data['menu_pre_proc'].str.contains(favourite_food))
    

    # Combine conditions with logical OR
    if conditions and len(filtered_data) != 0:
        filtered_data = restaurant_data[pd.concat(conditions, axis=1).any(axis=1)]
    else:
        filtered_data = restaurant_data 
    

    # Ranking - sort by ratings or other criteria
    restaurant_data = restaurant_data.sort_values(by='ratingValue', ascending=False)

    # Return top recommendations
    return restaurant_data.head(5) 


# llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')
# # agent= initialize_agent( [get_restaurant_info],
# #     llm,
# #     agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
# #     handle_parsing_errors=True,
# #     verbose = False)

# # tools = [get_restaurant_info, get_recommendation]
# # # Get the prompt to use - you can modify this!
# # prompt = hub.pull("hwchase17/openai-tools-agent")
# # agent = create_openai_tools_agent(llm, tools, prompt)
# # agent_executor =  AgentExecutor(agent=agent, tools=tools, verbose=True)

# # result = agent.invoke({'input': "Tell me about restaurant 'Chutneys' in Albufeira"})
# result = agent("Tell me about restaurant 'Chutneys' in Albufeira")
# print(result)

NameError: name 'agent' is not defined

In [75]:
instructions2 = {
                '[INSTRUCTION: Identification]': 
                 {'instruction description': f"""Greet the user by their username or first name, if it exists (is different from "No Identification Provided"). \
                                        Otherwise, greet the user as "Fellow Foodie". Introduce yourself as Filomena - FlavourFlix' virtual assistance. 
                                        \Ask the user what they need your help with. """ + identification_vars,
                    'when to use': "When the user greets the ChatBot and the content of the last message from the assistant is empty."},


                 '[INSTRUCTION: Question]':
                 {'instruction description': f"""Answer the user's question based on the provided context and chat history.""", 
                  'when to use': """For non-restaurant-related questions (e.g., about FlavourFlix or the virtual assistant)""" },

                  '[INSTRUCTION: Restaurant Description]': 
                  {'instruction description': f"""Find the restaurant with the closest name of the query in the data and \
                   return its description using the function get_information.""",
                    'when to use': """When the user inquires about a specific restaurant by name."""}
                                        }


instruction_identifier = """TASK: Your job is to assign an Instruction Identifier based on a user input (query)  \
    and a chat history. Consider the following instructions: """ + str(instructions2) + """You will receive \
    a chat history between the ChatBot and the user, and a final query from the user. 
    You will return the answer in the following output:
    [Instruction: Instruction Identifier] | query"""


prompt_templates2 = {'Instructions': instructions2,
                     'Instruction Identification': instruction_identifier}

class GPT_Helper:
    def __init__(self,
        OPENAI_API_KEY: str,
        system_behavior: str="",
        #model="gpt-4",
        #model = "gpt-3.5-turbo"
        model = "gpt-4-0613"
    ):
        self.client = OpenAI(api_key=OPENAI_API_KEY)
        self.messages = []
        self.model = model

        if system_behavior:
            self.messages.append({
                "role": "system",
                "content": system_behavior
            })

           
    def get_completion(self, prompt, temperature=0.0):
        self.messages = []
        self.messages.append({"role": "user", "content": prompt})
        completion = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            temperature=temperature,
        )
        self.messages.append(
            {
                "role": "assistant",
                "content": completion.choices[0].message.content}
        )
        return completion.choices[0].message.content
    

    def get_instruction(self, query, chat_history):
        prompt = f"""
                The User Message is:{query}
                The Chat History is:{chat_history}
                """

        query_formatted = self.get_completion(prompt)
        if query not in query_formatted:
            query_formatted = f'{query_formatted} | {query}'
        return query_formatted

In [76]:
filomena_core = GPT_Helper(OPENAI_API_KEY=local_settings.OPENAI_API_KEY, system_behavior=prompt_templates2['Instruction Identification'])
query = 'Hello'
history = filomena_core.messages[0]['content']

filomena_core.get_instruction(query, history)

'[INSTRUCTION: Identification] | Hello'

In [77]:
filomena_core.messages

[{'role': 'user',
  'content': '\n                The User Message is:Hello\n                The Chat History is:TASK: Your job is to assign an Instruction Identifier based on a user input (query)      and a chat history. Consider the following instructions: {\'[INSTRUCTION: Identification]\': {\'instruction description\': \'Greet the user by their username or first name, if it exists (is different from "No Identification Provided").                                         Otherwise, greet the user as "Fellow Foodie". Introduce yourself as Filomena - FlavourFlix\\\' virtual assistance. \\n                                        \\\\Ask the user what they need your help with. No Identification Provided\', \'when to use\': \'When the user greets the ChatBot and the content of the last message from the assistant is empty.\'}, \'[INSTRUCTION: Question]\': {\'instruction description\': "Answer the user\'s question based on the provided context and chat history.", \'when to use\': \'For non-

In [78]:
history = [filomena_core.messages[0], {'role': 'assistant', 'content': ''}]

In [79]:
query = 'Who are FlavourFlix authors?'

filomena_core.get_instruction(query, history)

'[INSTRUCTION: Question] | Who are FlavourFlix authors?'

In [80]:
query = 'What is FlavourFlix purpose?'
history = filomena_core.messages

filomena_core.get_instruction(query, history)

'[INSTRUCTION: Question] | What is FlavourFlix purpose?'

In [81]:
query = 'Where is the restaurant Chutneys?'
history = filomena_core.messages

filomena_core.get_instruction(query, history)

'[INSTRUCTION: Restaurant Description] | Where is the restaurant Chutneys?'

In [94]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [105]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


tools = [get_word_length]

In [106]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [108]:
from langchain_community.tools.convert_to_openai import format_tool_to_openai_function

llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

In [109]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [110]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [111]:
agent_executor.invoke({"input": "How many letters in the word educa"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


{'input': 'How many letters in the word educa',
 'output': 'There are 5 letters in the word "educa".'}

In [112]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [113]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [114]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [115]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


> Entering new AgentExecutor chain...
No, "educa" is not a real word in English.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".')],
 'output': 'No, "educa" is not a real word in English.'}

In [1]:
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.chat_models import ChatOpenAI

In [14]:
restaraunt_Data = pd.read_csv('data/preprocessed_restaurant_data.csv')

In [13]:
@tool
def get_restaurant_info(restaurant_name):
    """Gathers all the restaurant information from the FlavourFlix restaurant database, based on the name of the restaurant."""
    restaurant_data = pd.read_csv('data\preprocessed_restaurant_data.csv')
    restaurant_data = restaurant_data[restaurant_data['name'] == restaurant_name]
    result = f"""Name: {restaurant_data["name"].values[0]} \n /
    Cuisine Type: {restaurant_data["cuisine"].values[0]} \n /
    Average Price: {restaurant_data["averagePrice"].values[0]} \n /
    Location: {restaurant_data["location"].values[0]} \n /
    Restaurant Style: {restaurant_data["style"].values[0]} \n /
    Restaurant Schedule: {restaurant_data["schedule"].values[0]}"""
    return result


def get_data_match(data, word, col_to_match):
    nlp = spacy.load("en_core_web_md")

    word_embedding = nlp(word).vector
    word_embedding = nlp('mexican').vector
    similarities = {}
    for token in list(data[col_to_match].unique()):
        token_embedding = nlp(token).vector
        similarities[token] = cosine_similarity([word_embedding], [token_embedding])[0][0]

    return max(similarities, key=similarities.get)


def filter_schedule(restaurants, dinner_hour = None, lunch_hour = None):
    def contains_time_interval(schedule):
        if schedule in ['Closed', 'Not Available']:
            return False
        schedule_dict = {}
        try:
            schedule_dict = ast.literal_eval(schedule)
        except (SyntaxError, ValueError):
            pass

        for day_schedule in schedule_dict.values():
            if day_schedule == 'Closed':
                continue
            times = day_schedule.split(', ')
            for time in times:
                start, end = map(pd.to_datetime, time.split(' - '))
                if dinner_hour and lunch_hour:
                    if (start <= dinner_start <= end) or (start <= dinner_end <= end) or (start <= lunch_start <= end) or (start <= lunch_end <= end):
                        return True
                elif dinner_hour:
                    if (start <= dinner_start <= end) or (start <= dinner_end <= end):
                        return True
                elif lunch_hour:
                    if (start <= lunch_start <= end) or (start <= lunch_end <= end):
                        return True
                else:
                    return False
        return False

    dinner_start, dinner_end = map(pd.to_datetime, dinner_hour.split(' - '))
    lunch_start, lunch_end = map(pd.to_datetime, lunch_hour.split(' - '))

    filtered_restaurants = []
    for index, row in restaurants.iterrows():
        if contains_time_interval(row['schedule']):
            filtered_restaurants.append(row['name'])

    return filtered_restaurants

@tool
def get_recommendation(nationality=None, city=None, travel_car=None,  favourite_food=None,  restaurant_style=None,
                       cuisine_type=None, lunch_hour=None, dinner_hour=None,
                       normal_price_range=None):
    """Recommends restaurants based on the user's preferences and requirements."""
    restaurant_data = pd.read_csv('data/preprocessed_restaurant_data.csv')
    filtered_data = restaurant_data.copy()

    conditions = []

    #Location filters such that the restaurant is near the user and the place the user wants to go
    if city:
        city_match = get_data_match(restaurant_data, city, 'location')
        filtered_data = filtered_data[filtered_data['location'] == city_match]
        latitude, longitude = find_coordinates(city_match)
        if latitude is not None and longitude is not None:
            desired_location = Location(latitude, longitude)
            filtered_data = nearYou(desired_location, filtered_data)
            if travel_car:
                filtered_data['minutes_away'] = filtered_data.apply(lambda row: desired_location.getDirections(row['latitude'], row['longitude'], ['driving'])['driving'].minutes, axis=1)
                filtered_data = filtered_data[filtered_data['minutes_away'] <= 35]
            else:
                filtered_data['minutes_away'] = filtered_data.apply(lambda row: desired_location.getDirections(row['latitude'], row['longitude'], ['walking'])['walking'].minutes, axis=1)
                filtered_data = filtered_data[filtered_data['minutes_away'] <= 59]

    #Time filters such that the restaurant is open at the time the user wants to go
    if lunch_hour:
        filtered_data = filter_schedule(filtered_data, dinner_hour, lunch_hour)
    if dinner_hour:
        filtered_data = filter_schedule(filtered_data, dinner_hour, lunch_hour)
    
    #Additional filters
    if nationality and len(filtered_data) != 0:
        nationality_match = get_data_match(restaurant_data, nationality, 'cuisine')
        conditions.append(restaurant_data['cuisine'] == nationality_match)
    
    if restaurant_style:
        restaurant_style_match = get_data_match(restaurant_data, restaurant_style, 'style')
        conditions.append(restaurant_data['style'] == restaurant_style_match)
    
    if cuisine_type:
        cuisine_type_match = get_data_match(restaurant_data, cuisine_type, 'cuisine')
        conditions.append(restaurant_data['cuisine'] == cuisine_type_match)

    if normal_price_range:
        conditions.append(restaurant_data['averagePrice'] <= normal_price_range+4)

    if favourite_food:
        if detect(favourite_food) == 'en':
            conditions.append(restaurant_data['menu_en'].str.contains(favourite_food))
        else:
            conditions.append(restaurant_data['menu_pre_proc'].str.contains(favourite_food))
    

    # Combine conditions with logical OR
    if conditions and len(filtered_data) != 0:
        filtered_data = restaurant_data[pd.concat(conditions, axis=1).any(axis=1)]
    else:
        filtered_data = restaurant_data 
    

    # Ranking - sort by ratings or other criteria
    restaurant_data = restaurant_data.sort_values(by='ratingValue', ascending=False)

    restaurant_data = restaurant_data.head(5)
    # Return top recommendations
    return restaurant_data['name'].values.tolist()


In [23]:
import re

text = "No Identification Provided"

pattern = r"Username:\s*([^\s|]+)"
matches = re.search(pattern, text)

if matches:
    username = matches.group(1)

In [29]:
type(None) == type(matches)

True

In [8]:
tools = [get_recommendation]

In [9]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/structured-chat-agent")

In [10]:
# Choose the LLM that will drive the agent
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106")

# Construct the JSON agent
agent = create_structured_chat_agent(llm, tools, prompt)

In [11]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [12]:
agent_executor.invoke({"input": "cuisine: mexican, city: Albufeira, normal_price_range: 20, favourite_food: massa. Give me a restaurant recommendation"})



> Entering new AgentExecutor chain...
Thought: The user is looking for a Mexican restaurant in Albufeira with a normal price range of 20 and their favorite food is "massa."

Action:
```
{
  "action": "get_recommendation",
  "action_input": {
    "nationality": "Mexican",
    "city": "Albufeira",
    "normal_price_range": 20,
    "favourite_food": "massa"
  }
}
```                                                    url  \
1324  https://www.thefork.com/restaurant/rei-da-gula...   
224   https://www.thefork.com/restaurant/caravela-wy...   
372   https://www.thefork.com/restaurant/alegre-me-h...   
217   https://www.thefork.com/restaurant/grandbar-lo...   
419   https://www.thefork.com/restaurant/casa-do-arc...   

                                           name  \
1324                                Rei da Gula   
224            Caravela - Wyndham Grand Algarve   
372                    Alegre-me Healthy Brunch   
217   Grandbar & Lounge - Wyndham Grand Algarve   
419             Casa d

{'input': 'cuisine: mexican, city: Albufeira, normal_price_range: 20, favourite_food: massa. Give me a restaurant recommendation',
 'output': "I recommend the restaurant Caravela - Wyndham Grand Algarve in Almancil, Portugal. It offers International cuisine, has an average price of 21, and may have options that suit your preference for 'massa'. You can find more information and make a reservation at https://www.thefork.com/restaurant/caravela-wyndham-grand-algarve"}

In [1]:
from functions.chat_bot_v2 import *

2024-01-06 20:50:22.443 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


In [2]:
from functions.filomena_utils_v2 import *

In [3]:
fil = Filomena()

In [4]:
import os 
files = [f'text_data/{file}' for file in os.listdir('text_data')]

In [5]:
fil.initialize(files= files)

In [6]:
fil.generate_response('I want you to recommend me a restaurant, please')

"Oh, that's great to hear that you belong to the food personality The Adventurer! So, can you let me know which city you are in so that I can give you a restaurant recommendation that suits your taste?"

In [7]:
fil.generate_response('Yes. I am currently in Lisbon!')

'I recommend the following restaurants in Lisbon for The Adventurer:\n\n1. Seisan - Japanese cuisine\n   - Average price: €30\n   - Ambience rating: 9.5\n   - Food rating: 9.8\n   - Service rating: 9.7\n   - Address: R. da Madalena 271, Lisbon, Portugal\n   - [More info](https://www.thefork.com/restaurant/seisan-r728776)\n\n2. KatanaSushi (Santos) - Japanese cuisine\n   - Average price: €17\n   - Ambience rating: 9.2\n   - Food rating: 9.7\n   - Service rating: 9.5\n   - Address: R. Presidente Arriaga nº 55, Lisbon, Portugal\n   - [More info](https://www.thefork.com/restaurant/katanasushi-santos-r728776)\n\n3. Ajitama Ramen Bistro - Alecrim - Japanese cuisine\n   - Average price: €19\n   - Ambience rating: 9.3\n   - Food rating: 9.7\n   - Service rating: 9.4\n   - Address: R. do Alecrim 47A, 1200-014 Lisboa, Portugal\n   - [More info](https://www.thefork.com/restaurant/ajitama-ramen-bistro-alecrim-r728776)\n\n4. IZANAGI SushiCafé - Japanese cuisine\n   - Average price: €20\n   - Ambien

In [8]:
fil.generate_response('Describe them to me, please')

'Sure! Bruno Moreira is a tech enthusiast with a background in data science. Carolina Shaul is a food lover who brings her expertise in gastronomy to the team. Guilherme Carriço is a technology expert who contributes his skills to the development of FlavourFlix. Madalena Frango is a passionate foodie who helps curate the restaurant recommendations on the platform. Together, they have created FlavourFlix to combine their love for technology and food, providing users with the best restaurant recommendations.'

In [9]:
fil.generate_response('Describe me Guilherme')

'Guilherme Carriço is the Quality Assurance Specialist at FlavourFlix. He plays a crucial role in guaranteeing the flawless functionality of the platform, ensuring a seamless and enjoyable experience for users. His commitment to maintaining the highest standards is instrumental in the success of FlavourFlix.'

In [10]:
fil.generate_response("Talk to me about the origins of each of Flavourflix' founders")

"Bruno Moreira, one of the founders of FlavourFlix, has a background in data science and graduated from NOVA Information Management School in Lisbon with a bachelor's degree in data science. His analytical mindset and coding skills have been instrumental in shaping FlavourFlix's business strategy.\n\nCarolina Shaul, another founder of FlavourFlix, brings her expertise in gastronomy to the team. Raised in a family that cherished both traditional and modern cuisine, she found herself naturally drawn to the intersection of technology and gastronomy. Carolina's warm and organized approach as a project manager and developer has been invaluable to the FlavourFlix team.\n\nGuilherme Carriço, the Quality Assurance Specialist at FlavourFlix, is a technology expert who plays a crucial role in guaranteeing the flawless functionality of the platform. His commitment to maintaining the highest standards ensures a seamless and enjoyable experience for users.\n\nUnfortunately, there is no specific inf

In [10]:
fil.generate_response("I am having trouble the 'Current Location' functionality on the app. Can you help me?")

'To troubleshoot the \'Current Location\' functionality on the app, you can try the following steps:\n\n1. Check your internet connection to ensure it is stable.\n2. Refresh the app by closing it and reopening it.\n3. Make sure you have an account and are logged in correctly.\n4. Verify that you are using the Chrome browser, as this functionality is only available for Chrome.\n5. Check if you have location services and pop-ups disabled on your browser or device. If so, enable them.\n6. Confirm if you are in an accessible location for the app to determine your current location accurately.\n\nIf the error persists, it is recommended to report the issue to technical support through the app page "Contact Us" or directly via the FlavourFlix email: flavour.flixx@gmail.com. They will be able to assist you further.'

In [11]:
fil.generate_response("How does FlavourFlix use my data?")

'FlavourFlix collects your data with the purpose of enhancing your user experience on the platform. The data collected is used to refine and personalize the restaurant recommendations provided to you. It is also used to power the analytics dashboard, which helps FlavourFlix gain insights into user preferences and trends. Rest assured that all data handling adheres to stringent privacy and security standards, ensuring a safe and enjoyable experience for our valued users.'

In [12]:
fil.generate_response("Explain to me what is a Pastél de Nata?")

'A Pastél de Nata is a traditional Portuguese pastry that is also known as a Portuguese custard tart. It consists of a crispy, flaky pastry shell filled with a creamy custard made from eggs, sugar, cream, and vanilla. The top of the tart is usually caramelized, giving it a deliciously sweet and slightly burnt flavor. Pastéis de Nata are typically enjoyed with a sprinkle of cinnamon on top and are a popular treat in Portugal and around the world.'

In [13]:
fil.generate_response("Describe the typical Portuguese cuisine in 5 words.")

'Some words that describe typical Portuguese cuisine are: flavorful, diverse, fresh, hearty, and seafood-rich.'

In [14]:
fil.generate_response("Summarize one of FlavourFlix' blog posts.")

'The main topic of one of FlavourFlix\'s blog posts is "Tailored Tastes: Unveiling Your Food Personality with FlavourFlix" by Fernando Santos on January 20, 2023.'

In [15]:
fil.generate_response("What are the FlavourFlix food personalities? Describe them to me.")

" Absolutely! FlavourFlix offers five distinct food personalities to help users navigate the vast world of dining choices. Here they are:\n\n1. Comfort Food Lover: This food personality is for those who enjoy classic, comforting dishes that evoke a sense of nostalgia and warmth. Think hearty stews, mac and cheese, and indulgent desserts.\n\n2. Conscious Eater: If you prioritize organic, sustainable, and locally sourced ingredients, the Conscious Eater food personality is perfect for you. FlavourFlix will recommend restaurants that align with your values and introduce you to new and exciting options.\n\n3. Fine Dining Connoisseur: For those who appreciate the finer things in life, the Fine Dining Connoisseur food personality is tailored to your refined taste. Expect recommendations for upscale restaurants with exquisite cuisine and impeccable service.\n\n4. Low-Cost Foodie: If you're on a budget but still want to enjoy delicious meals, the Low-Cost Foodie food personality is ideal. Flav

In [16]:
fil.generate_response("Who sings the song 'Thriller'?")

"I'm sorry, but I don't have information about the artist behind the song 'Thriller'. My area of expertise is FlavourFlix and culinary-related topics. If you have any questions about FlavourFlix or Portuguese gastronomy, I'll be happy to help!"

In [17]:
fil.generate_response('What is the capital of Portugal?')

"The capital of Portugal is Lisbon. It is a vibrant and historic city known for its beautiful architecture, delicious cuisine, and lively atmosphere. If you're planning a visit to Portugal, Lisbon is definitely a must-see destination!"

In [24]:
fil.generate_response('Tell me more about the restaurant Chutneys')

"The restaurant 'Chutneys' is an Indian restaurant located in Albufeira, with an average price of 14. Its restaurant style is Festivities and it is open from Monday to Sunday."

In [25]:
fil.generate_response('What do you know about the restaurant a Galinha da Vizinha?')

"'A Taberna da Praça' is a Portuguese restaurant located in Braga. It has an average price of 17 and is closed every day of the week."